In [23]:
import pandas as pd
import numpy as np

In [24]:
xl = pd.ExcelFile("Lifespan_MCUDN_E1F1.xlsx")

Dict_Excel_Sheets = {
    sheets: xl.parse(sheets, nrows=3, header=None, index_col=0).T.set_index("Days")
    for sheets in xl.sheet_names
}

In [25]:
def convert_to_lifespan_readable(
    DF, Col_Head="Unnamed", Col_Num_Dead="Total Deaths", Col_Num_Cens="Total Escape"
):
    """
    Input: Takes DataFrame and string Col_Head
    Functon: Converts counts dead and censors into binary format for survival curve (e.g. 11100)
    Output: DataFrame with string Col_Head as column title.
    """

    if Col_Num_Dead and Col_Num_Cens not in DF.columns:
        return print(f"'{Col_Num_Dead}' and '{Col_Num_Cens}' are not columns in DF")
    
    Binary_Num_Dead_Cens = np.empty(0)
    BinaryNoDays = np.empty(0)

    for index in DF.index:
        Number_Dead = DF.loc[index, Col_Num_Dead]
        Number_Cens = DF.loc[index, Col_Num_Cens]
        Number_Dead_Cens = np.append(np.ones(Number_Dead), np.zeros(Number_Cens))
        Binary_Num_Dead_Cens = np.append(Binary_Num_Dead_Cens, Number_Dead_Cens)
        BinaryNoDays = np.append(
            BinaryNoDays, np.repeat(index, Number_Dead + Number_Cens)
        )

    DF_Life = pd.DataFrame(data=Binary_Num_Dead_Cens, index=BinaryNoDays, dtype="int32")
    DF_Life.columns = [Col_Head]
    return DF_Life

In [26]:
def dict_concat_2_LifespanDF(Dict_Excel_Sheets):
    """
    Input: Takes Dictionary of excel sheet dataframes.
    Functon: 
    Output: 
    """

    DF = pd.DataFrame()

    for keys in Dict_Excel_Sheets:
        Temp_DF = convert_to_lifespan_readable(
            Dict_Excel_Sheets.get(keys), Col_Head=keys
        )
        DF = pd.concat([DF, Temp_DF], sort=True)

    return DF

In [27]:
DF_Lifespan = dict_concat_2_LifespanDF(Dict_Excel_Sheets)

In [30]:
DF_Lifespan.filter(regex=("MCU")).dropna(how="all").to_clipboard()

In [21]:
DF_Lifespan.filter(regex=("MCUDN")).to_clipboard()

In [15]:
DF_Lifespan["MCUDN. RU486"].to_clipboard()

/Users/morriso1/opt/anaconda3/envs/ImageAnalysis/lib/python3.7/site-packages/pandas/io/clipboards.py:117: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  obj.to_csv(buf, sep=sep, encoding="utf-8", **kwargs)


In [13]:
convert_to_lifespan_readable(Dict_Excel_Sheets.get('MCUDN. RU486'))

,Unnamed
3.0,1
3.0,1
6.0,1
6.0,1
8.0,1
...,...
98.0,1
98.0,1
101.0,1
101.0,1


In [29]:
DF_Lifespan

,LacZ RNAi. Norm Food,LacZ RNAi. RU486,MCUDN. Norm Food,MCUDN. RU486
3.0,1.0,NaN,NaN,NaN
3.0,1.0,NaN,NaN,NaN
3.0,1.0,NaN,NaN,NaN
3.0,1.0,NaN,NaN,NaN
6.0,1.0,NaN,NaN,NaN
...,...,...,...,...
98.0,NaN,NaN,NaN,1.0
98.0,NaN,NaN,NaN,1.0
101.0,NaN,NaN,NaN,1.0
101.0,NaN,NaN,NaN,1.0
